# create stats for paper

In [36]:
import sys
sys.path.append('../../')
import glob
import shutil
from pathlib import Path
import os

import numpy as np
import pandas as pd
from config import CLEAN_DATA_DIR, OUTPUT_DIR, RAW_DATA_DIR
import db as db
from scripts.normalize_taxa import add_normalized_name_column

from scripts.normalize_data import (
    check_duplicate_columns
)

from scripts.shared_utils import (
    create_df_from_db_rows, 
    log_df
)

In [37]:
clean_data_path = CLEAN_DATA_DIR

metadata_file = OUTPUT_DIR/'metadata'/'LIMS'/'Micropal_changes.csv' 
# metadata_file = OUTPUT_DIR/'metadata'/'LIMS'/'Micropal_changes_4.csv' 


date = '2022-11-15'

taxa_list_file = OUTPUT_DIR/'taxa'/'LIMS'/f"taxa_list_{date}.csv"
taxa_crosswalk_list_file = OUTPUT_DIR/'taxa'/'LIMS'/f"taxa_crosswalk_{date}.csv"

PI_file = RAW_DATA_DIR/'PI_processed_files'/f'LIMS_Micropal_headers_PBDB_Taxonomy_notes_taxa_list_{date}.csv'
PI_4_file = OUTPUT_DIR /'taxa'/'LIMS'/f'PI_Micropal_CSV_4_normalized_taxa_list_with_pbdb_{date}.csv'
additional_taxa_path = OUTPUT_DIR/'taxa'/'LIMS'/f'addtional_species_{date}.csv'
PI_NOAA_file = OUTPUT_DIR/'taxa'/'NOAA'/f'taxa_crosswalk_{date}.csv'

all_columns_file = OUTPUT_DIR/'tmp/all_LIMS_taxa_columns.csv'


## LIMS verbatim taxa names 

In [38]:
sql = """
select count(*) as count
from taxa_crosswalk
where taxon_id in (select taxon_id from samples_taxa)
"""

row = db.fetch_one(sql)
print(  row['count'])
# 5378

5378


## LIMS taxa verbatim names grouped by taxon group

In [39]:
sql = """
select count(*) as count, taxon_group 
from taxa_crosswalk
where taxon_id in (select taxon_id from samples_taxa)
group by taxon_group
order by taxon_group;
"""

sum  = 0

rows = db.fetch_all(sql)
for row in rows:
    print(row['taxon_group'],  row['count'])
    sum  += row['count']

benthic_forams 1709
bolboformids 2
chrysophyte_cysts 1
diatoms 746
dinoflagellates 65
ebridians 10
nannofossils 953
ostracods 21
other 15
palynology 144
planktic_forams 1029
radiolarians 643
silicoflagellates 40


In [40]:
sum

5378

## LIMS taxa names

if taxa is in multiple taxon group, count each taxa

In [41]:
sql = """
select count(*) as count 
from taxa 
where id in (select taxon_id from samples_taxa);
"""

row = db.fetch_one(sql)
print(  row['count'])

# 4736

4692


if taxa is in multiple taxon group, count as one taxa

In [42]:
sql = """
select count(distinct(name)) as count 
from taxa 
where id in (select taxon_id from samples_taxa);
"""

row = db.fetch_one(sql)
print(  row['count'])

# 4712

4667


##  LIMS taxa names grouped by taxon group

In [43]:
sql = """
select count(*) as count, taxon_group 
from taxa
where id in (select taxon_id from samples_taxa)
group by taxon_group
order by taxon_group;
"""
sum  = 0

rows = db.fetch_all(sql)
for row in rows:
    print(row['taxon_group'],  row['count'])
    sum  += row['count']

benthic_forams 1516
bolboformids 2
chrysophyte_cysts 1
diatoms 664
dinoflagellates 60
ebridians 7
nannofossils 819
ostracods 20
other 11
palynology 133
planktic_forams 837
radiolarians 596
silicoflagellates 26


In [44]:
sum

4692

## distinct LIMS taxon name above genus

if taxa is in multiple taxon group, count as one taxa

In [60]:
sql = """
select count(distinct(taxon_name_above_genus))
from taxa
where id in (select taxon_id from samples_taxa)
and taxon_name_above_genus is not null;
"""

row = db.fetch_one(sql)
print( row['count'])
# 86

86


## distinct LIMS genus

if taxa is in multiple taxon group, count as one taxa

In [61]:
sql = """
select count(distinct(genus_name))
from taxa
where id in (select taxon_id from samples_taxa)
and genus_name is not null;
"""
row = db.fetch_one(sql)
print( row['count'])
# 1068

1068


## distinct LIMS species

if taxa is in multiple taxon group, count as one taxa

In [62]:
sql = """
select count(distinct(name))
from taxa
where id in (select taxon_id from samples_taxa)
and species_name is not null
and species_name not in ('sp.', 'spp.');
"""
row = db.fetch_one(sql)
print( row['count'])
# 3732

3732


## distinct LIMS subspecies

if taxa is in multiple taxon group, count as one taxa

In [59]:
sql = """
select count(distinct(name))
from taxa
where id in (select taxon_id from samples_taxa)
and subspecies_name is not null;
"""
row = db.fetch_one(sql)
print( row['count'])
# 136

136


## nontaxa values in taxa files sent to PIs

### verbatim taxa names from LIMS, LIMS 4, additional species, NOAA

In [49]:
lims_df = pd.read_csv(PI_file, header=9)
lims_df = lims_df.drop(0)
lims_df.dropna(axis=0, how='all', inplace=True)
add_normalized_name_column(lims_df)

log_df(lims_df, 2)

(4742, 33)


,taxon_group,verbatim_name,name,"name to use (if different from ""name"")",name comment field,Comment,Notes (change to Internal only notes?),Any taxon above genus,genus modifier,genus name,...,family_taxon_name,order_taxon_id,order_taxon_name,class_taxon_id,class_taxon_name,phylum_taxon_id,phylum_taxon_name,kingdom_taxon_id,kingdom_taxon_name,normalized_name
1,benthic_forams,Pyrite,Pyrite,NaN,NaN,"not a taxa; Going into Macrostrat, not PBDB",Summer 2020: deal with later; should go in as ...,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,
2,benthic_forams,Euuvigerina miozea (group) >100 m,Euuvigerina miozea (group) >100 m,Euuvigerina miozea,NaN,group,"Summer 2020: enter ""Euuvigerina miozea"" and re...",NaN,NaN,Euuvigerina,...,NaN,NaN,NaN,NaN,NaN,288974.0,Foraminifera,212476.0,Rhizaria,Euuvigerina miozea


In [50]:
lims_4_df = pd.read_csv(PI_4_file)
lims_4_df.dropna(axis=0, how='all', inplace=True)
add_normalized_name_column(lims_4_df)

log_df(lims_4_df, 2)

(695, 34)


,taxon_group,verbatim_name,Name,Comment,Notes (change to Internal only notes?),Any taxon above genus,genus modifier,genus name,subgenera modifier,subgenera name,...,class_taxon_id,class_taxon_name,phylum_taxon_id,phylum_taxon_name,kingdom_taxon_id,kingdom_taxon_name,unranked clade_taxon_id,unranked clade_taxon_name,abbrev_genus,normalized_name
0,benthic_forams,Agglutinated Foram. gen. et sp. indet.,Textulariia indet.,NaN,NaN,Textulariia indet.,NaN,NaN,NaN,NaN,...,NaN,NaN,288974.0,Foraminifera,212476.0,Rhizaria,NaN,NaN,False,Textulariia indet.
1,benthic_forams,Agglutinated indet.,Textulariia indet.,NaN,NaN,Textulariia indet.,NaN,NaN,NaN,NaN,...,NaN,NaN,288974.0,Foraminifera,212476.0,Rhizaria,NaN,NaN,False,Textulariia indet.


In [51]:
add_df = pd.read_csv(additional_taxa_path)
add_df.dropna(axis=0, how='all', inplace=True)

log_df(add_df, 2)

(30, 38)


,sample_id,file,sample,taxon_group,verbatim_name,code,Name,Comment,Notes (change to Internal only notes?),Any taxon above genus,...,class_taxon_id,class_taxon_name,phylum_taxon_id,phylum_taxon_name,kingdom_taxon_id,kingdom_taxon_name,unranked clade_taxon_id,unranked clade_taxon_name,normalized_name,path
0,203528,374_U1525A_radiolarians.csv,374-U1525A-1H-1-IW_MUDLINE,radiolarians,additional species: Lonchosphaera spicata,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,4,Radiolaria,212476,Rhizaria,NaN,NaN,Lonchosphaera spicata,LIMS/Micropal_CSV_2/374_U1525A_radiolarians.csv
1,203528,374_U1525A_radiolarians.csv,374-U1525A-1H-1-IW_MUDLINE,radiolarians,additional species: Poulpus spp.-like,NaN,Poulpus spp.,NaN,NaN,NaN,...,NaN,NaN,4,Radiolaria,212476,Rhizaria,NaN,NaN,Poulpus spp.,LIMS/Micropal_CSV_2/374_U1525A_radiolarians.csv


In [52]:
noaa_df = pd.read_csv(PI_NOAA_file)
noaa_df.dropna(axis=0, how='all', inplace=True)

log_df(noaa_df, 2)

(7763, 40)


,taxon_group,verbatim_name,name,Comment,Notes (change to Internal only notes?),Any taxon above genus modifier,Any taxon above genus,genus modifier,genus name,subgenera modifier,...,order_taxon_name,class_taxon_id,class_taxon_name,phylum_taxon_id,phylum_taxon_name,kingdom_taxon_id,kingdom_taxon_name,unranked clade_taxon_id,unranked clade_taxon_name,normalized_name
0,benthic_forams,Abyssamina incisa,NaN,NaN,NaN,NaN,NaN,NaN,Abyssamina,NaN,...,Rotaliida,428504.0,Globothalamea,288974.0,Foraminifera,212476.0,Rhizaria,NaN,NaN,Abyssamina incisa
1,benthic_forams,Adercotryma glomeratum,NaN,NaN,NaN,NaN,NaN,NaN,Adercotryma,NaN,...,NaN,NaN,NaN,288974.0,Foraminifera,212476.0,Rhizaria,NaN,NaN,Adercotryma glomeratum


In [53]:
cols = ['verbatim_name', 'taxon_group', 'normalized_name']

lims_df_2 = lims_df[cols]
lims_4_df_2 = lims_4_df[cols]
add_df_2 = add_df[cols]

combine_lims = pd.concat([lims_df_2, lims_4_df_2, add_df_2])
combine_lims.loc[combine_lims['normalized_name'] == '', 'normalized_name'] = np.nan
combine_lims = combine_lims.drop_duplicates()

combine_lims.shape

(5317, 3)

### non-taxa names

In [54]:
nontaxa_df = combine_lims[combine_lims['normalized_name'].isna()]
nontaxa_df

,verbatim_name,taxon_group,normalized_name
1,Pyrite,benthic_forams,NaN
21,fossil,benthic_forams,NaN
22,fossil_group,benthic_forams,NaN
3068,Preservation palynofacies,palynology,NaN
3082,Exotic,palynology,NaN
3196,Organic matter,planktic_forams,NaN
3197,Terrestrial organic matter,planktic_forams,NaN
3213,Pyrite,planktic_forams,NaN
3215,ADDITIONAL SPECIES,planktic_forams,NaN
4125,ADDITIONAL SPECIES,radiolarians,NaN


In [55]:
nontaxa_df.shape

(22, 3)

### normalized taxa names

In [22]:
combine_lims2 = combine_lims[['normalized_name', 'taxon_group']]
combine_lims2 = combine_lims2.dropna(subset=['normalized_name'])

combine_lims2 = combine_lims2.drop_duplicates() 
log_df(combine_lims2)

(4660, 2)


,normalized_name,taxon_group
2,Euuvigerina miozea,benthic_forams
3,Euuvigerina rodleyi,benthic_forams
4,Foraminifera indet.,benthic_forams
5,Pleurostomellidae indet.,benthic_forams
6,Ostracoda indet.,benthic_forams


In [23]:
sql = """
select name
from taxa 
where id in (select taxon_id from samples_taxa);
"""

names = set()
rows = db.fetch_all(sql)
for row in rows:
    names.add(row['name'])

In [24]:
len(names)

4712

In [25]:
set(combine_lims2['normalized_name']) - names

{'Gephyrocapsa spp. (large)', 'Hemiaulus danicus'}

In [26]:
names - set(combine_lims2['normalized_name']) - set(noaa_df['normalized_name'])

set()

In [27]:
noaa_taxa = names - set(combine_lims2['normalized_name'])
len(noaa_taxa)

78

# create taxa summary reports

## all verbatim names

In [29]:
sql = """
select  
taxa.name, taxa.taxon_group, taxa.pbdb_taxon_id,
taxa_crosswalk.verbatim_name,
taxa_crosswalk.comment
from taxa 
join taxa_crosswalk on taxa.id = taxa_crosswalk.taxon_id
where   taxa.id in (select taxon_id from samples_taxa)
order by taxa.name, taxa.taxon_group, taxa_crosswalk.verbatim_name,  taxa_crosswalk.comment;
"""

rows = db.fetch_all_dict(sql)
len(rows)
# 5377

5378

In [30]:
df = create_df_from_db_rows(rows)
df['pbdb_taxon_id'] = df['pbdb_taxon_id'].astype(pd.Int64Dtype())
df.head()

,name,taxon_group,pbdb_taxon_id,verbatim_name,comment
0,"""Amorphous organic matter""",dinoflagellates,<NA>,Amorphous organic matter,not a taxa name
1,"""Amorphous organic matter""",palynology,<NA>,Amorphous organic matter,not a taxa name
2,"""Black phytoclasts""",dinoflagellates,<NA>,Black phytoclasts,not a taxa name
3,"""Black phytoclasts""",palynology,<NA>,Black phytoclasts,not a taxa name
4,"""Black woody phytoclasts""",palynology,<NA>,Black woody phytoclasts,not a taxa name


In [31]:
# df.to_csv(OUTPUT_DIR/'taxa'/'LIMS'/f'taxa_list_entries_{date}.csv', index=False)

## all names; combine multiple verbatim names and comments

In [32]:
sql = """
select  
taxa.name, taxa.taxon_group, taxa.pbdb_taxon_id,
array_to_string(array_agg(taxa_crosswalk.verbatim_name), '; ') as verbatim_name, 
array_to_string(array_agg(taxa_crosswalk.comment), '; ') as comment
from taxa 
join taxa_crosswalk on taxa.id = taxa_crosswalk.taxon_id
where   taxa.id in (select taxon_id from samples_taxa)
group by taxa.name, taxa.taxon_group , taxa.pbdb_taxon_id
order by taxa.name, taxa.taxon_group,  verbatim_name, comment ;
"""

rows = db.fetch_all_dict(sql)
len(rows)

# 4735

4736

In [33]:
df = create_df_from_db_rows(rows)
df['pbdb_taxon_id'] = df['pbdb_taxon_id'].astype(pd.Int64Dtype())
df.head()

,name,taxon_group,pbdb_taxon_id,verbatim_name,comment
0,"""Amorphous organic matter""",dinoflagellates,<NA>,Amorphous organic matter,not a taxa name
1,"""Amorphous organic matter""",palynology,<NA>,Amorphous organic matter,not a taxa name
2,"""Black phytoclasts""",dinoflagellates,<NA>,Black phytoclasts,not a taxa name
3,"""Black phytoclasts""",palynology,<NA>,Black phytoclasts,not a taxa name
4,"""Black woody phytoclasts""",palynology,<NA>,Black woody phytoclasts,not a taxa name


In [34]:
# df.to_csv(OUTPUT_DIR/'taxa'/'LIMS'/f'taxa_list_distinct_taxonomic_entries_{date}.csv', index=False)

## all names; combine multiple verbatim names, comments, and taxon groups

In [30]:
sql = """
select count(distinct taxon_id) from samples_taxa;
"""

db.fetch_one(sql)[0]

4692

In [31]:
sql = """
select count(distinct(taxa.name, taxa.taxon_group))
from taxa 
where taxa.id in (select taxon_id from samples_taxa);
"""

db.fetch_one(sql)[0]

4692

In [32]:
sql = """
select count(distinct(taxa.name))
from taxa 
where taxa.id in (select taxon_id from samples_taxa);
"""

db.fetch_one(sql)[0]

4667

In [33]:
sql = """
select  
array_to_string(array_agg(taxa.taxon_group ORDER BY taxa.taxon_group), '; ') as taxon_group, 
array_to_string(array_agg(taxa_crosswalk.verbatim_name ORDER BY taxa_crosswalk.verbatim_name), '; ') as verbatim_name, 
taxa.name,
array_to_string(array_agg(taxa_crosswalk.comment ORDER BY taxa_crosswalk.comment), '; ') as comment,
taxon_name_above_genus,
genus_modifier, genus_name,
subgenera_modifier, subgenera_name,
species_modifier, species_name,
subspecies_modifier, subspecies_name,
non_taxa_descriptor,
array_to_string(array_agg(taxa_crosswalk.comments ORDER BY taxa_crosswalk.comments), '; ') as comments,
taxa.pbdb_taxon_id, taxa.pbdb_taxon_rank, taxa.pbdb_taxon_name
from taxa 
join taxa_crosswalk on taxa.id = taxa_crosswalk.taxon_id
where   taxa.id in (select taxon_id from samples_taxa)
group by taxa.name,  taxon_name_above_genus,
genus_modifier, genus_name,
subgenera_modifier, subgenera_name,
species_modifier, species_name,
subspecies_modifier, subspecies_name,
non_taxa_descriptor,  
taxa.pbdb_taxon_id, taxa.pbdb_taxon_rank, taxa.pbdb_taxon_name
order by taxa.name,   verbatim_name, comment ;
"""




rows = db.fetch_all_dict(sql)
len(rows)

# 4667

4667

In [34]:
df = create_df_from_db_rows(rows)
df['pbdb_taxon_id'] = df['pbdb_taxon_id'].astype(pd.Int64Dtype())
df.tail()

,taxon_group,verbatim_name,name,comment,taxon_name_above_genus,genus_modifier,genus_name,subgenera_modifier,subgenera_name,species_modifier,species_name,subspecies_modifier,subspecies_name,non_taxa_descriptor,comments,pbdb_taxon_id,pbdb_taxon_rank,pbdb_taxon_name
4662,nannofossils,Zygodiscus plectopons,Zygodiscus plectopons,<NA>,<NA>,<NA>,Zygodiscus,<NA>,<NA>,<NA>,plectopons,<NA>,<NA>,<NA>,<NA>,87678,genus,Zygodiscus
4663,nannofossils,Zygrhablithus bijugatus,Zygrhablithus bijugatus,<NA>,<NA>,<NA>,Zygrhablithus,<NA>,<NA>,<NA>,bijugatus,<NA>,<NA>,<NA>,<NA>,416732,species,Zygrhablithus bijugatus
4664,radiolarians,Spongurus cf. elliptica,cf. Spongurus elliptica,<NA>,<NA>,cf.,Spongurus,<NA>,<NA>,<NA>,elliptica,<NA>,<NA>,<NA>,<NA>,328,genus,Spongurus
4665,radiolarians,Thyrsocyrtis cf. norrisi,cf. Thyrsocyrtis norrisi,<NA>,<NA>,cf.,Thyrsocyrtis,<NA>,<NA>,<NA>,norrisi,<NA>,<NA>,<NA>,<NA>,685,genus,Thyrsocyrtis
4666,radiolarians,Velicucullus sp. cf. oddgurneri,cf. Velicucullus oddgurneri,<NA>,<NA>,cf.,Velicucullus,<NA>,<NA>,<NA>,oddgurneri,<NA>,<NA>,<NA>,<NA>,705,genus,Velicucullus


In [35]:
df.to_csv(OUTPUT_DIR/'taxa'/'LIMS'/f'taxa_list_distinct_taxonomic_entries_across_groups_{date}.csv', index=False)

## taxa where PIs add quotes to name

taxa name with quotes where verbatim name does not have quotes

In [38]:
sql = """
select distinct taxa.name,
taxa.taxon_group,  
array_to_string(array_agg(taxa_crosswalk.verbatim_name), '; ') as verbatim_names
from taxa 
join taxa_crosswalk on taxa.id = taxa_crosswalk.taxon_id
where taxa.name like '"%'
and taxa_crosswalk.verbatim_name  not like '"%'
and taxa.id in (select taxon_id from samples_taxa)
group by taxa.name, taxa.taxon_group
order by taxa.name;
"""

rows = db.fetch_all_dict(sql)
len(rows)

34

In [39]:
df = create_df_from_db_rows(rows)
df.head()

,name,taxon_group,verbatim_names
0,"""Amorphous organic matter""",dinoflagellates,Amorphous organic matter
1,"""Amorphous organic matter""",palynology,Amorphous organic matter
2,"""Black phytoclasts""",dinoflagellates,Black phytoclasts
3,"""Black phytoclasts""",palynology,Black phytoclasts
4,"""Black woody phytoclasts""",palynology,Black woody phytoclasts


## Nontaxa fields

In [53]:
df = pd.read_csv(all_columns_file, dtype=str, usecols=['column'])
all_columns = set(df['column'])
len(all_columns)
# 5556

5556

In [54]:
lims_df = pd.read_csv(taxa_crosswalk_list_file)
lims_verbatim_names = set(lims_df['verbatim_name'])
len(lims_verbatim_names)
# 5368

5368

In [55]:
lims_verbatim_names - all_columns

{'Calocycletta virginis.1',
 'Discoaster loeblichii\nDiscoaster prepentaradiatus',
 'Gephyrocapsa (large)',
 'Hemiaulus danicus'}

In [56]:
nontaxa = all_columns - lims_verbatim_names 

cleaned = {
'Label ID', 'Sample', 
'Top [cm]', 'Top offset [cm]', 'Top[cm] [cm]',
'Bottom [cm]', 'Bottom offset [cm]', 'Bottom[cm] [cm]',
'Top Depth [m]', 'Top Depth[m] [m]', 'Top depth [m]',
'Bottom Depth [m]', 'Bottom Depth[m] [m]', 'Bottom depth [m]',
    
'Top [cm]', 'Top (cm)', 'Interval (top)', 'Interval Top (cm) on SHLF', 
'Top interval (cm)', 'top interval (cm)', 'top (cm)',
    
'Bottom [cm]', 'Bottom (cm)', 'Interval (bottom)', 'Interval Bot (cm) on SHLF', 
'Bottom interval (cm)', 'bottom interval (cm)', 'bottom (cm)',
    
'Depth (csf)', 'Top depth CSF-B (m):', 'Top Depth (CSF m)', 'Depth top CSF-A (m)',
'Top Depth [CFS m]', 'Top Depth [CSF m]', 'Top depth [CSF m]', 'top depth', 
'Top CSF-A (m)', 'Top Depth (m)', 'depth CSF-A (m)', 'Top Depth [m]',
'Depth CSF-A (m)', 'Depth CSF (m)', 'depth Top (m)', 'depth Top (m CSF-A)',
'Top Depth CSF-A (m)', 'depth CSF-A Top (m)', 'Top Depth (m) CSF-A', 
'depth Top CSF-A (m)', 'Top Depth CFS (m)', 'depth CSF-A', 'Top depth CSF (m)',
'Top (m CSF-A)', 'Top depth CSF-B (m)', 'Depth m (m csf)', 'Depth (m) CSF-A',
    
'Bottom depth CSF-B (m):', 'Depth bottom CSF-A (m)', 'Bottom Depth [CFS m]', 
'bottom depth', 'Bottom CSF-A (m)', 'Bottom Depth (m)', 'Bottom Depth [m]', 
'depth Bottom (m)', 'depth Bottom (m CSF-A)', 'Bottom Depth CSF-A (m)', 
'depth CSF-A Bottom (m)', 'Bottom Depth (m) CSF-A', 'depth Bottom CSF-A (m)', 
'Bottom (m CSF-A)', 'Bottom depth CSF-B (m)'
}

ignore = {
'Unnamed: 13',
'Unnamed: 14',
'Unnamed: 179',
'Unnamed: 21',
'Unnamed: 81',
'Dextral:Sinistral _N. acostaensis_',
'Dextral:Sinistral _P. finalis_',
'Dextral:Sinistral _P. obliquiloculata_',
'Dextral:Sinistral _P. praecursor_',
'Dextral:Sinistral _P. praespectabilis_',
'Dextral:Sinistral _P. primalis_',
'Dextral:Sinistral _P. spectabilis_',
'ADDITIONAL SPECIES',
}

print(len(nontaxa))

nontaxa = nontaxa.union(cleaned)
print(len(nontaxa))

nontaxa = nontaxa - ignore
print(len(nontaxa))

# 192
# 253
# 240

192
253
240


In [57]:
# nontaxa

In [58]:
df = pd.DataFrame(nontaxa, columns=['field'])
df = df.sort_values('field')
df.head()


,field
145,% Planktic Foraminifera within whole sample
173,342-U1408A-2H-2-W 100/102-FORAM
135,A/W
155,Abundance
81,Abundance (%)


In [59]:
df.to_csv(OUTPUT_DIR/'tmp'/f'nontaxa_{date}.csv',  index=False)
